This notebook uses keras to train on the body of sonnets written by Shakespeare. In the end, we will use the model we train to write some sonnet with the help of Shakespeare himself!

# Preprocessing

In [1]:
# write a small function to read the text file as a giant string
def read_file(filepath):
    with open(filepath) as file:
        str_text = file.read()
        
        return str_text

In [3]:
sonnet = read_file('sonnet.txt')

In [8]:
len(sonnet)

98331

In [12]:
#sonnet

## Tokenization

There are many ways to do this but here we will use spaCy.

In [49]:
import spacy
from keras.preprocessing.text import Tokenizer
import numpy as np

Using TensorFlow backend.


In [7]:
# disable a few function to make our process faster
nlp = spacy.load('en', disable = ['parser', 'tagger', 'ner'])

In [9]:
# avoid later complaints 
nlp.max_length = 98331

In [43]:
# avoid NN overfit common punctuations
def separate_punc(doc_text):
    return [token.text.strip().lower() for token in nlp(doc_text) if token.text not in '1234567890!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\n\n\s']

In [62]:
tokens = separate_punc(sonnet)

In [63]:
tokens = list(filter(lambda x: x != '', tokens))

In [65]:
# make a text_sequences which is a giant list of all the possible sequences
train_len = 10 + 1

text_sequences = []

for i in range(train_len, len(tokens)):
    seq = tokens[i - train_len:i]
    text_sequences.append(seq)

## Transform the tokens to numerica values

In [66]:
#Format this to a numeric system that keras can understand
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [69]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [77]:
# we have encode the original word sequences to numeric sequences
print(sequences[0])
print(text_sequences[0])

[36, 502, 1349, 180, 232, 633, 8, 1348, 45, 11, 352]
['from', 'fairest', 'creatures', 'we', 'desire', 'increase', 'that', 'thereby', 'beauty', "'s", 'rose']


In [76]:
# check to confirm if the id matches the word
for i in sequences[0]:
    print(f'{i}:{tokenizer.index_word[i]}')

36:from
502:fairest
1349:creatures
180:we
232:desire
633:increase
8:that
1348:thereby
45:beauty
11:'s
352:rose


In [78]:
vocab_size = len(tokenizer.word_counts)

In [80]:
# this is the number of unique tokens
vocab_size

3240

In [83]:
# cast the sequences as numpy array 
sequences = np.array(sequences)

# Create the LSTM model

We are going to use the begining of the sequences as features and the last word as target we are going to predict.

## Feature, target extraction

In [99]:
from keras.utils import to_categorical 
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [87]:
sequences.shape

(18080, 11)

In [88]:
# grabbing the sequences except for the last element of each sequence
X = sequences[:, :-1]
y = sequences[:, -1]

In [90]:
# one extra space to hold 0
y = to_categorical(y, num_classes = vocab_size + 1)

In [129]:
X.shape, y.shape

((18080, 10), (18080, 3241))

## Build the model

In [118]:
# the seq_len is the length of a general sentence
def create_model(vocab_size, seq_len):
    model = Sequential()
    model.add(Embedding(vocab_size, seq_len, input_length = seq_len))
    model.add(LSTM(50, return_sequences = True))
    model.add(LSTM(50))
    model.add(Dense(50, activation = 'relu'))
    model.add(Dense(vocab_size, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    print(model.summary())
    return model

In [126]:
# the length of the sentence is no more than 10 words
model = create_model(vocab_size = vocab_size + 1, seq_len = 10)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 10, 10)            32410     
_________________________________________________________________
lstm_5 (LSTM)                (None, 10, 50)            12200     
_________________________________________________________________
lstm_6 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_5 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_6 (Dense)              (None, 3241)              165291    
Total params: 232,651
Trainable params: 232,651
Non-trainable params: 0
_________________________________________________________________
None


## Fit model

In [127]:
from pickle import dump, load

In [128]:
model.fit(X, y, batch_size = 128, epochs = 200, verbose = 2)

/Users/liwenhuang/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/200
18080/18080 [==============================] - 3s 148us/step - loss: 6.9654 - accuracy: 0.0220
Epoch 2/200
18080/18080 [==============================] - 2s 108us/step - loss: 6.4659 - accuracy: 0.0263
Epoch 3/200
18080/18080 [==============================] - 2s 106us/step - loss: 6.4433 - accuracy: 0.0259
Epoch 4/200
18080/18080 [==============================] - 2s 105us/step - loss: 6.4281 - accuracy: 0.0264
Epoch 5/200
18080/18080 [==============================] - 2s 106us/step - loss: 6.4094 - accuracy: 0.0258
Epoch 6/200
18080/18080 [==============================] - 2s 105us/step - loss: 6.3927 - accuracy: 0.0261
Epoch 7/200
18080/18080 [==============================] - 2s 107us/step - loss: 6.3717 - accuracy: 0.0262
Epoch 8/200
18080/18080 [==============================] - 2s 107us/step - loss: 6.3377 - accuracy: 0.0254
Epoch 9/200
18080/18080 [==============================] - 2s 106us/step - loss: 6.2792 - accuracy: 0.0259
Epoch 10/200
18080/18080 [===========

Epoch 77/200
18080/18080 [==============================] - 3s 156us/step - loss: 3.7965 - accuracy: 0.1654
Epoch 78/200
18080/18080 [==============================] - 3s 152us/step - loss: 3.7772 - accuracy: 0.1678
Epoch 79/200
18080/18080 [==============================] - 3s 151us/step - loss: 3.7352 - accuracy: 0.1787
Epoch 80/200
18080/18080 [==============================] - 3s 153us/step - loss: 3.7199 - accuracy: 0.1769
Epoch 81/200
18080/18080 [==============================] - 3s 152us/step - loss: 3.7028 - accuracy: 0.1870
Epoch 82/200
18080/18080 [==============================] - 3s 150us/step - loss: 3.6757 - accuracy: 0.1855
Epoch 83/200
18080/18080 [==============================] - 3s 148us/step - loss: 3.6719 - accuracy: 0.1871
Epoch 84/200
18080/18080 [==============================] - 3s 143us/step - loss: 3.6462 - accuracy: 0.1923
Epoch 85/200
18080/18080 [==============================] - 3s 140us/step - loss: 3.6318 - accuracy: 0.1918
Epoch 86/200
18080/18080 [==

In [130]:
model.save('sonnet_writer_v1.h5')

In [131]:
# save the tokenizer as well
dump(tokenizer, open('sonnet_tokenizer', 'wb'))

# Ready to write some sonnets!

In [136]:
from keras.preprocessing.sequence import pad_sequences

In [165]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    
    input_text = seed_text
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen = seq_len, truncating = 'pre')
        pred_word_idx = model.predict_classes(pad_encoded, verbose = 0)[0]
        pred_word = tokenizer.index_word[pred_word_idx]
        input_text += " " + pred_word
        output_text.append(pred_word)
        
    return ' '.join(output_text)

In [198]:
seed_text = 'Wine'
need_len = 10 - len(seed_text.split())
generate_text(model, tokenizer, 10, seed_text, need_len)

"a number imperfect all fiend o thought 's verse"

With my AI Shakespeare, I have produced the following sonnet:


I shall not tell thee your eyes or thence, 

thou are mischievous years pass o but i then say.

Give me a pen moon her reasons 's civil, 

Those that money cant buy a all brow.

Why is the god damn bird still singing censures retention!

What shall I say no yellow and privilege in I!

My muse enlighten me,  thorns but fears came silvered his!

There I lie and die a better excuse so invent, 

Come as dust and go as her winters placed after. 

Wine a number imperfect all fiend o thought 's verse.